<a href="https://colab.research.google.com/github/Praz98/HomeLLC/blob/main/Feature_Engineering_HomeLLC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import scipy
import matplotlib.pyplot as plt
#importing libraries

%matplotlib inline


In [24]:
final_df = pd.read_csv("/content/drive/MyDrive/cleaned data/merged_data.csv")
final_df.rename(columns = {"Unnamed: 0":"DATE"},inplace=True)
final_df.set_index(final_df["DATE"],inplace=True)
final_df.drop(columns = ["DATE"],inplace=True)

In [25]:
dfn = final_df.copy()

In [26]:
dfn.columns

Index(['avg_expenditure_55_64', 'avg_expenditure_35_44',
       'avg_expenditure_25_34', 'avg_expenditure_45_54', 'CPI',
       'employees_construction', 'HCAI_GOVT', 'houses-for-sale-to-sold',
       'HCAI_PP', 'HCAI_GSE', 'MORTGAGE30US', 'GDP', 'PERMIT',
       'pvt_owned_house_under_const', 'personal_saving_rate', 'population',
       'house_units_completed', 'retail_sales_home_furnishing_stores',
       'UNRATE', 'unrate_construction', 'industrial_production_cement',
       'producer_price_index_concrete_brick', 'job_gains_private',
       'residential_const_val', 'target', 'homeownership',
       'nonresidential_const_val'],
      dtype='object')

In [27]:
# Log transformation to remove outliers
dfn["unrate_construction"] = np.log(dfn["unrate_construction"])
dfn["houses-for-sale-to-sold"] = np.log(dfn["houses-for-sale-to-sold"])


dfn["avg_expenditure_35_54"] = dfn["avg_expenditure_35_44"] + dfn["avg_expenditure_45_54"]

# Adding GDP per capita
dfn["gdp_per_capita"] = dfn["GDP"] / dfn["population"]

In [28]:
# Adding a categorical variable to gauge CPI TREND
dfn["CPI_TREND"] = dfn["CPI"].apply(lambda x : "UP" if x > 0 else "DOWN")
dfn["CPI_TREND"].value_counts()

UP      180
DOWN     72
Name: CPI_TREND, dtype: int64

In [29]:
dfn["GDP_RATE"] = np.log(dfn["GDP"]/dfn["GDP"].shift(12))

In [30]:
# Adding a categorical variable to gauge GDP TREND
dfn["GDP_TREND"] = dfn["GDP_RATE"].apply(lambda x : "UP" if x > 0 else "DOWN")

In [31]:
dfn["EMP_CONST_RATE"] = np.log(dfn["employees_construction"]/dfn["employees_construction"].shift(12))

In [32]:
# Adding a categorical variable to gauge trend in number of construction employees
dfn["EMP_CONST_TREND"] = dfn["EMP_CONST_RATE"].apply(lambda x : "UP" if x > 0 else "DOWN")

In [33]:
# Since HCAI_GOVT, HCAI_GSE AND HCAI_PP are highly collinear with each other, we are linearly combining them
dfn["HCAI"] = (dfn["HCAI_GOVT"] + dfn["HCAI_GSE"] + dfn["HCAI_PP"])/3

In [34]:
dfn["houses-for-sale-to-sold-rate"] = np.log(dfn["houses-for-sale-to-sold"]/dfn["houses-for-sale-to-sold"].shift(12))

In [35]:
dfn["HOUSES_S2S_TREND"] = dfn["houses-for-sale-to-sold-rate"].apply(lambda x : "UP" if x > 0 else "DOWN")

In [36]:
for column in dfn.columns:
    try:
        temp_rate = np.log(dfn[column]/dfn[column].shift(12)).dropna()
        temp_cum = dfn[column].cumsum()

        temp_rate_cum = np.log(temp_cum/temp_cum.shift(12)).dropna()
        print()
        print(column,round(dfn.corr()["target"][column],2))
        trate12 = np.corrcoef(dfn["target"][12:],temp_rate)[0,1]
        print(f"Correlation between target variable and 12 months change of rate {round(trate12,2)}")

        tcum = (np.corrcoef(dfn["target"],temp_cum)[0,1])
        print(f"correlation cumulative {round(tcum,2) }")

        tcumrate = np.corrcoef(dfn["target"][12:],temp_rate_cum)[0,1]
        print(f"correlation cumulative rate {round(tcumrate,2) }")



    except:
        continue


avg_expenditure_55_64 0.88
Correlation between target variable and 12 months change of rate -0.12
correlation cumulative 0.79
correlation cumulative rate -0.55

avg_expenditure_35_44 0.92
Correlation between target variable and 12 months change of rate 0.2
correlation cumulative 0.79
correlation cumulative rate -0.54

avg_expenditure_25_34 0.9
Correlation between target variable and 12 months change of rate 0.26
correlation cumulative 0.79
correlation cumulative rate -0.55

avg_expenditure_45_54 0.88
Correlation between target variable and 12 months change of rate 0.06
correlation cumulative 0.8
correlation cumulative rate -0.55

CPI -0.04

employees_construction 0.5
Correlation between target variable and 12 months change of rate 0.24
correlation cumulative 0.79
correlation cumulative rate -0.53

HCAI_GOVT -0.43
Correlation between target variable and 12 months change of rate 0.07
correlation cumulative 0.78
correlation cumulative rate -0.54

houses-for-sale-to-sold 0.23
Correlation 

<ipython-input-36-04c6f8634144>:8: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(column,round(dfn.corr()["target"][column],2))
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


HCAI -0.4
Correlation between target variable and 12 months change of rate 0.09
correlation cumulative 0.77
correlation cumulative rate -0.54

houses-for-sale-to-sold-rate 0.14


In [37]:
dfn["industrial_production_cement_cum"] = dfn["industrial_production_cement"].cumsum()
dfn["cpi_cum"] = dfn["CPI"].cumsum()

dfn["houses-for-sale-to-sold-cum"] = dfn["houses-for-sale-to-sold"].cumsum()
dfn["job_gains_private_cum"] = dfn["job_gains_private"].cumsum()
dfn["pvt_owned_house_under_const_cum"] = dfn["pvt_owned_house_under_const"].cumsum()

dfn["PERMIT_cum"] = dfn["PERMIT"].cumsum()

dfn["house_units_completed_cum"] = dfn["house_units_completed"].cumsum()




## Rate of change of industrial_production_cement_cum
dfn["industrial_production_cement_rate"] = np.log(dfn["industrial_production_cement_cum"]/dfn["industrial_production_cement_cum"].shift(12))

dfn["pvt_owned_house_under_const_rate"] = np.log(dfn["pvt_owned_house_under_const_cum"]/dfn["pvt_owned_house_under_const_cum"].shift(12))

dfn["PERMIT_rate"] = np.log(dfn["PERMIT_cum"]/dfn["PERMIT_cum"].shift(12))

dfn["job_gains_private_rate"] = np.log(dfn["job_gains_private_cum"]/dfn["job_gains_private_cum"].shift(12))


dfn["pvt_owned_house_under_const_rate"] = np.log(dfn["pvt_owned_house_under_const_cum"]/dfn["pvt_owned_house_under_const_cum"].shift(12))


dfn["house_units_completed_rate"] = np.log(dfn["house_units_completed_cum"]/dfn["house_units_completed_cum"].shift(12))


dfn["job_gains_private_rate"] = np.log(dfn["job_gains_private_cum"]/dfn["job_gains_private_cum"].shift(12))


dfn["job_gains_private_rate"] = np.log(dfn["job_gains_private_cum"]/dfn["job_gains_private_cum"].shift(12))


In [38]:
dfn["PERMIT_TREND"] =  dfn["PERMIT_rate"].apply(lambda x : "UP" if x > 0 else "DOWN")
dfn["job_gains_private_trend"] =  dfn["job_gains_private_rate"].apply(lambda x : "UP" if x > 0 else "DOWN")
dfn["pvt_owned_house_under_const_trend"] =  dfn["pvt_owned_house_under_const_rate"].apply(lambda x : "UP" if x > 0 else "DOWN")

dfn["house_units_completed_trend"] =  dfn["house_units_completed_rate"].apply(lambda x : "UP" if x > 0 else "DOWN")

In [39]:
dfn.corr()["target"][abs(dfn.corr()["target"])>0.5]
dfn.to_csv("/content/drive/MyDrive/cleaned data/merged_added_features.csv")

<ipython-input-39-87fc488f1de6>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  dfn.corr()["target"][abs(dfn.corr()["target"])>0.5]


In [40]:

# Load your DataFrame from the file (assuming it's already loaded)
# If not, replace this with your code to load the DataFrame
dfn = pd.read_csv("/content/drive/MyDrive/cleaned data/merged_added_features.csv")

# Drop the column "houses-for-sale-to-solde"
#dfn.drop(columns=["houses-for-sale-to-solde"], inplace=True)

# Save the modified DataFrame to a CSV file
dfn.to_csv("/content/drive/MyDrive/cleaned data/merged_added_features.csv", index=False)


In [41]:
df = pd.read_csv("/content/drive/MyDrive/cleaned data/merged_added_features.csv")
df.columns

Index(['DATE', 'avg_expenditure_55_64', 'avg_expenditure_35_44',
       'avg_expenditure_25_34', 'avg_expenditure_45_54', 'CPI',
       'employees_construction', 'HCAI_GOVT', 'houses-for-sale-to-sold',
       'HCAI_PP', 'HCAI_GSE', 'MORTGAGE30US', 'GDP', 'PERMIT',
       'pvt_owned_house_under_const', 'personal_saving_rate', 'population',
       'house_units_completed', 'retail_sales_home_furnishing_stores',
       'UNRATE', 'unrate_construction', 'industrial_production_cement',
       'producer_price_index_concrete_brick', 'job_gains_private',
       'residential_const_val', 'target', 'homeownership',
       'nonresidential_const_val', 'avg_expenditure_35_54', 'gdp_per_capita',
       'CPI_TREND', 'GDP_RATE', 'GDP_TREND', 'EMP_CONST_RATE',
       'EMP_CONST_TREND', 'HCAI', 'houses-for-sale-to-sold-rate',
       'HOUSES_S2S_TREND', 'industrial_production_cement_cum', 'cpi_cum',
       'houses-for-sale-to-sold-cum', 'job_gains_private_cum',
       'pvt_owned_house_under_const_cum', 'PER